# Loan Prediction - To find out if someone is eligible for the loan given input

In [1]:
from azureml import Workspace

ws = Workspace()
ds = ws.datasets['loadPredict-Train.csv']
df = ds.to_dataframe()

### Import Libraries

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df.shape

(614, 13)

In [5]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

### Checking for missing values

In [6]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

### Imputing Categorical columns with mode and numerical with mean

In [7]:
#categorical
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df['Married'].fillna(df['Married'].mode()[0], inplace=True)
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
#numerical
df['LoanAmount'].fillna(df['LoanAmount'].mean(), inplace=True)

In [8]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [9]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [10]:
y = df['Loan_Status']

## Encoding categorical variables

In [11]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

In [12]:
y = encode.fit(y).transform(y)

In [13]:
category = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status'] 
encoder = LabelEncoder()
for i in category:
  df[i] = encoder.fit_transform(df[i])

In [14]:
df.dtypes

Loan_ID               object
Gender                 int64
Married                int64
Dependents             int64
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
Loan_Status            int64
dtype: object

In [15]:
df.drop(['Loan_ID','Loan_Status'],axis = 1,inplace=True)

In [16]:
x = df

### Splitting the dataset to perform training and testing

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1237)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((491, 11), (123, 11), (491,), (123,))

### Modelling

In [18]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression()

In [19]:
clf1.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
yhat = clf1.predict(x_test)

In [21]:
clf1.score(x_train,y_train)

0.81262729124236255

In [22]:
clf1.score(x_test,y_test)

0.80487804878048785

In [23]:
# Lets try few ensemble methods
from sklearn.tree import DecisionTreeClassifier 
clf2=DecisionTreeClassifier(max_depth=4)

In [24]:
clf2.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [25]:
clf2.score(x_train,y_train)

0.82281059063136452

In [26]:
clf2.score(x_test,y_test)

0.81300813008130079

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf3 = RandomForestClassifier()
clf3.fit(x_train,y_train)
clf3.score(x_train,y_train)

0.98981670061099791

In [28]:
clf3.score(x_test,y_test)

0.75609756097560976

### Grid Search for better hyperparameter tuning

In [32]:
# Lets try Grid Search for Random Forest

from sklearn.model_selection import GridSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num =5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num =4)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'max_depth': [10, 43, 76, 110, None], 'bootstrap': [True, False], 'min_samples_leaf': [1, 2], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'n_estimators': [200, 650, 1100, 1550, 2000]}


In [33]:
gs_clf3 = GridSearchCV(estimator = clf3, param_grid = random_grid, cv =3,scoring='accuracy', verbose=2, n_jobs = -1)

In [34]:
gs_clf3.fit(x_train,y_train)

Fitting 3 folds for each of 600 candidates, totalling 1800 fits
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_estimators=200 
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_estimators=200 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_estimators=200, total=   0.5s
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_estimators=200, total=   0.5s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_estimators=200 
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_estimators=650 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_estimators=200, total=   0.4s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=2, max_features=auto, n_est

[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=5, max_features=auto, n_estimators=2000, total=   4.3s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=650 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=650, total=   1.3s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1100 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=650, total=   1.3s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1100 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1100, total=   2.3s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1100 
[CV]  max_depth=10, boot

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   42.4s


[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1100, total=   2.1s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1550 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1550, total=   3.2s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=1550 
[CV] max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=2000 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=1, min_samples_split=10, max_features=auto, n_estimators=2000, total=   3.9s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=2, max_features=auto, n_estimators=200 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=2, max_features=auto, n_estimators=200, total=   0.4s
[CV] max_depth=10, boots

[CV] max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=10, max_features=auto, n_estimators=200 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=10, max_features=auto, n_estimators=200, total=   0.4s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=10, max_features=auto, n_estimators=200 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=10, max_features=auto, n_estimators=200, total=   0.5s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=10, max_features=auto, n_estimators=650 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=10, max_features=auto, n_estimators=650, total=   1.2s
[CV] max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=10, max_features=auto, n_estimators=650 
[CV]  max_depth=10, bootstrap=True, min_samples_leaf=2, min_samples_split=5, max_features=auto, n_estimators=2000, total=   4.0s
[CV] max_depth=10, bootstrap

...

[CV] max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=2, max_features=sqrt, n_estimators=2000 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=2, max_features=sqrt, n_estimators=1550, total=   3.2s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=2, max_features=sqrt, n_estimators=2000 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=2, max_features=sqrt, n_estimators=2000, total=   4.2s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=2, max_features=sqrt, n_estimators=2000 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=2, max_features=sqrt, n_estimators=2000, total=   4.2s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=5, max_features=sqrt, n_estimators=200 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=1, min_samples_split=5, max_features=sqrt, n_estimators=200, total=   0.4s
[CV] m

[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=200 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=200, total=   0.4s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=200 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=200, total=   0.4s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=200 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=200, total=   0.4s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=650 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=2, max_features=sqrt, n_estimators=650, total=   1.3s
[CV] max_dep

[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=10, max_features=sqrt, n_estimators=650 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=10, max_features=sqrt, n_estimators=650, total=   1.4s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=10, max_features=sqrt, n_estimators=650 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=5, max_features=sqrt, n_estimators=2000, total=   4.4s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=10, max_features=sqrt, n_estimators=650 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=10, max_features=sqrt, n_estimators=650, total=   1.4s
[CV] max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=10, max_features=sqrt, n_estimators=1100 
[CV]  max_depth=None, bootstrap=False, min_samples_leaf=2, min_samples_split=10, max_features=sqrt, n_estimators=650, total=   1.4s
[CV

[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed: 38.0min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [10, 43, 76, 110, None], 'bootstrap': [True, False], 'min_samples_leaf': [1, 2], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'n_estimators': [200, 650, 1100, 1550, 2000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=2)

In [35]:
accuracy = gs_clf3.best_score_

In [36]:
accuracy

0.81466395112016299

In [38]:
gs_clf3.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 200}

In [41]:
classifier = RandomForestClassifier(bootstrap=True,max_depth=10,max_features='auto',min_samples_leaf=1,min_samples_split=10,n_estimators=200)

In [42]:
classifier.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [44]:
yhat =classifier.predict(x_test)

In [45]:
classifier.score(x_train,y_train)

0.85132382892057024

In [46]:
classifier.score(x_test,y_test)

0.80487804878048785

### Metrics to see how the model performed

In [48]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(yhat,y_test))
print(confusion_matrix(yhat,y_test))

             precision    recall  f1-score   support

          0       0.43      0.84      0.57        19
          1       0.97      0.80      0.87       104

avg / total       0.88      0.80      0.83       123

[[16  3]
 [21 83]]


In [49]:
# Since the accuracy is not that great, lets try scaling the outliers. 
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [56]:
# Scale the data seperate to avoid data leakage 
x_train = scale.fit(x_train).transform(x_train)
x_test = scale.fit(x_test).transform(x_test)

In [53]:
classifier.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [54]:
classifier.score(x_train,y_train)

0.84928716904276991

In [55]:
classifier.score(x_test,y_test)

0.80487804878048785

In [57]:
yhat = classifier.predict(x_test)

In [58]:
print(classification_report(yhat,y_test))
print(confusion_matrix(yhat,y_test))

             precision    recall  f1-score   support

          0       0.41      0.88      0.56        17
          1       0.98      0.79      0.88       106

avg / total       0.90      0.80      0.83       123

[[15  2]
 [22 84]]


#### This model would predict if someone is eligible for a loan or not. We could work futher on the dataset to increase its accuracy.